## <strong>CÁC HỆ MẬT HIỆN ĐẠI</strong>

### <strong>DES</strong>

DES (Data Encryption Standard) là chuẩn mã hóa dữ liệu đầu tiên trên thế giới, do Cơ quan an ninh Quốc gia Hoa Kỳ (NSA) đề xuất trên cơ sở cải tiến thuật toán Lucifer do hãng IBM công bố năm 1964. DES đã được sử dụng rộng rãi ở Hoa Kỳ và nhiều quốc gia khác trong các thập kỷ 70, 80, 90 cho đến khi được thay thế bởi Tiêu chuẩn mã hóa dữ liệu tiên tiến AES (Advanced Encryption Standard) vào năm 2002.

DES là một mã khối, chỉ ra như hình dưới đây

![Caesar Cipher K=3](../img/des_fig1.png)

Đầu vào của DES là khối 64 bit, đầu ra cũng là khối 64 bit. Khóa mã hóa có độ dài 56 bit, nhưng thực chất ban đầu là 64 bit, được lấy đi các bit ở vị trí chia hết cho 8 dùng để kiểm tra tính chẵn lẻ.

Quá trình mã hóa được tạo thành bởi 2 phép hoán vị P - box, đây là những phép hoán vị khởi tạo và kết thúc, và gồm 16 chu kỳ (vòng) Feistel

#### <strong>Chuyển key 64 bit thành 56 bit (qua bảng PC1)</strong>

Khóa ban đầu là 1 xâu có độ dài 64 bit, bit thứ 8 của mỗi byte sẽ được lấy ra để kiểm tra phát hiện lỗi, tạo ra chuỗi 56 bit. Sau khi bỏ các bit kiểm tra ta sẽ hoán vị chuỗi 56 bit này. Hai bước trên được thực hiện thông qua hoán vị ma trận PC-1 (Permuted choice 1) như hình dưới đây.

![PC1](../img/pc1.jpg)



In [1]:

PC1 = [
    57, 49, 41, 33, 25, 17, 9,  1, 
    58, 50, 42, 34, 26, 18, 10, 2,  
    59, 51, 43, 35, 27, 19, 11, 3,  
    60, 52, 44, 36, 63, 55, 47, 39, 
    31, 23, 15, 7,  62, 54, 46, 38, 
    30, 22, 14, 6,  61, 53, 45, 37, 
    29, 21, 13, 5,  28, 20, 12, 4
]

def hex_to_bits(hex_string):
    bits = bin(int(hex_string, 16))[2:].zfill(64)  
    return [int(bit) for bit in bits]

def apply_pc1(key_64):
    return [key_64[i - 1] for i in PC1]

def bits_to_hex(bits):
    return hex(int(''.join(str(bit) for bit in bits), 2))[2:].zfill(14).upper()

def generate_56_bit_key(key_hex):
    key_64 = hex_to_bits(key_hex)  
    key_56_bits = apply_pc1(key_64)
    return key_56_bits

key_hex = "133457799BBCDFF1" 
key_56_bits = generate_56_bit_key(key_hex)
key_56_hex = bits_to_hex(key_56_bits)
print(f"Khóa 56 bit (sau khi áp dụng PC-1): {key_56_hex}")

Khóa 56 bit (sau khi áp dụng PC-1): F0CCAAF556678F


#### <strong>Key 56 bit tách ra LK, RK và thực hiện dịch tạo ra 16 key 56 bit</strong>


16 vòng lặp của DES chạy cùng thuật toán như nhau nhưng với 16 khóa con khác nhau. Các khóa con đều được sinh ra từ khóa chính của DES bằng thuật toán sinh khóa con. 

Tiếp theo ta kết quả sau khi PC-1 thành 2 phần : C0 : 28 bit đầu. D0 : 28 bit cuối. Mỗi phần sẽ được xử lý 1 cách độc lập. `Ci = LSi(Ci - 1) Di = LSi(Ci - 1)` với `1 <= i <= 16`. `LSi` là biểu diễn phép dịch bit vòng (cyclic shift) sang trái 1 hoặc 2 vị trí tùy thuộc vào i.

| Vòng lặp           | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 |
|--------------------|---|---|---|---|---|---|---|---|---|----|----|----|----|----|----|----|
| Số lần dịch trái   | 1 | 1 | 2 | 2 | 2 | 2 | 2 | 2 | 1 | 2  | 2  | 2  | 2  | 2  | 2  | 1  |


In [2]:
shift_schedule = [
    1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1
]

def left_shift(bits, shifts):
    return bits[shifts:] + bits[:shifts]

def bits_to_hex(bits):
    return hex(int(''.join(str(bit) for bit in bits), 2))[2:].zfill(14).upper()

def generate_subkeys(key_56):
    left, right = key_56[:28], key_56[28:]

    subkeys_56bit = [] 

    for round in range(16):
        left = left_shift(left, shift_schedule[round])
        right = left_shift(right, shift_schedule[round])

        combined_key = left + right
        subkeys_56bit.append(combined_key) 

    return subkeys_56bit

subkeys_56bit = generate_subkeys(key_56_bits)
print("\n16 khóa con 56 bit sau khi dịch:")
for i, subkey in enumerate(subkeys_56bit):
    subkey_hex = bits_to_hex(subkey)
    print(f"Subkey {i + 1} (56 bit): {subkey_hex}")


16 khóa con 56 bit sau khi dịch:
Subkey 1 (56 bit): E19955FAACCF1E
Subkey 2 (56 bit): C332ABF5599E3D
Subkey 3 (56 bit): 0CCAAFF56678F5
Subkey 4 (56 bit): 332ABFC599E3D5
Subkey 5 (56 bit): CCAAFF06678F55
Subkey 6 (56 bit): 32ABFC399E3D55
Subkey 7 (56 bit): CAAFF0C678F556
Subkey 8 (56 bit): 2ABFC339E3D559
Subkey 9 (56 bit): 557F8663C7AAB3
Subkey 10 (56 bit): 55FE199F1EAACC
Subkey 11 (56 bit): 57F8665C7AAB33
Subkey 12 (56 bit): 5FE19951EAACCF
Subkey 13 (56 bit): 7F866557AAB33C
Subkey 14 (56 bit): FE19955EAACCF1
Subkey 15 (56 bit): F866557AAB33C7
Subkey 16 (56 bit): F0CCAAF556678F


#### <strong>Từ mỗi key 56 bit trên, tạo ra key 48 bit qua bảng PC2 (tạo ra 16 key)</strong>

Cuối cùng sử dụng hoán vị cố định PC-2 (Permuted choice 2) để hoán vị chuỗi CiDi 56 bit tạo thành khóa Ki với 48 bit.

![PC1](../img/pc2.jpg)

In [3]:
PC2 = [
    14, 17, 11, 24, 1,  5,  3, 28, 
    15, 6,  21, 10, 23, 19, 12, 4, 
    26, 8,  16, 7,  27, 20, 13, 2,  
    41, 52, 31, 37, 47, 55, 30, 40, 
    51, 45, 33, 48, 44, 49, 39, 56, 
    34, 53, 46, 42, 50, 36, 29, 32
]

def apply_pc2(key_56):
    return [key_56[i - 1] for i in PC2]  

def bits_to_hex(bits):
    return hex(int(''.join(str(bit) for bit in bits), 2))[2:].zfill(12).upper()

def generate_48_bit_subkeys(subkeys_56bit):
    subkeys_48bit = []  
    for subkey in subkeys_56bit:
        subkey_48bit = apply_pc2(subkey)  
        subkeys_48bit.append(subkey_48bit)  
    return subkeys_48bit

subkeys_48bit = generate_48_bit_subkeys(subkeys_56bit)
print("\n16 khóa con 48 bit (sau khi áp dụng PC-2):")
for i, subkey in enumerate(subkeys_48bit):
    subkey_hex = bits_to_hex(subkey)
    print(f"Subkey {i + 1} (48 bit): {subkey_hex}")


16 khóa con 48 bit (sau khi áp dụng PC-2):
Subkey 1 (48 bit): 1B02EFFC7072
Subkey 2 (48 bit): 79AED9DBC9E5
Subkey 3 (48 bit): 55FC8A42CF99
Subkey 4 (48 bit): 72ADD6DB351D
Subkey 5 (48 bit): 7CEC07EB53A8
Subkey 6 (48 bit): 63A53E507B2F
Subkey 7 (48 bit): EC84B7F618BC
Subkey 8 (48 bit): F78A3AC13BFB
Subkey 9 (48 bit): E0DBEBEDE781
Subkey 10 (48 bit): B1F347BA464F
Subkey 11 (48 bit): 215FD3DED386
Subkey 12 (48 bit): 7571F59467E9
Subkey 13 (48 bit): 97C5D1FABA41
Subkey 14 (48 bit): 5F43B7F2E73A
Subkey 15 (48 bit): BF918D3D3F0A
Subkey 16 (48 bit): CB3D8B0E17F5


#### <strong>Quá trình mã hóa DES</strong>

Chia thành 3 giai đoạn:

<strong>Giai đoạn 1</strong>

Với bản rõ cho trước `x`, 1 xâu `x'` sẽ được tạo ra bằng cách hoán vị các bit của `x` theo hoán vị ban đầu IP. Tiếp theo `x'` sẽ được chia thành 2 phần `L0, R0`. `x' = IP(x) = L0R0` Trong đó `L0` là 32 bit đầu, `R0` là 32 bit cuối.

![PC1](../img/ip.jpg)

In [4]:
IP = [
    58, 50, 42, 34, 26, 18, 10, 2,  60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,  64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17, 9,  1,  59, 51, 43, 35, 27, 19, 11, 3,
    61, 53, 45, 37, 29, 21, 13, 5,  63, 55, 47, 39, 31, 23, 15, 7
]

def apply_ip(plain_text_64):
    return [plain_text_64[i - 1] for i in IP]

def bits_to_hex(bits):
    return hex(int(''.join(str(bit) for bit in bits), 2))[2:].zfill(8).upper()

def hex_to_bits(hex_string):
    bits = bin(int(hex_string, 16))[2:].zfill(64)  
    return [int(bit) for bit in bits]

def generate_lr_from_ip(plain_text_64):
    ip_result = apply_ip(plain_text_64)
    L0 = ip_result[:32]  
    R0 = ip_result[32:]  
    return ip_result, L0, R0

plain_text_hex = "0123456789ABCDEF"  
plain_text_64 = hex_to_bits(plain_text_hex)

ip_result, L0, R0 = generate_lr_from_ip(plain_text_64)

print(f"After IP: {bits_to_hex(ip_result)}")
print(f"L0: {bits_to_hex(L0)}")
print(f"R0: {bits_to_hex(R0)}")


After IP: CC00CCFFF0AAF0AA
L0: CC00CCFF
R0: F0AAF0AA


<strong>Giai đoạn 2</strong>

Tính toán 16 lần bằng 1 hàm xác định. Ta sẽ tính `Li`, `Ri` (`1 <=> i <=> 16`) theo quy tắc: `Li = Ri - 1`. `Ri = Li - 1 XOR f(Ri - 1, Ki)`. Với Ki là khóa được sinh ra ở quá trình tạo khóa, `f` là một hàm sẽ được trình bày ở phần sau.

![PC1](../img/encr.png)

Đầu vào hàm f có 2 biến:

- Biến thứ nhất: `Ri - 1` là xâu bit có độ dài 32 bit. 
- Biến thứ hai: Ki là xâu bit có độ dài 48 bit. Đầu ra của hàm f là xâu có độ dài 32 bit. 

Quy trình hoạt động của hàm f như sau:

- Biến thứ nhất `Ri - 1` được mở rộng thành một xâu có độ dài 48 bit theo một hàm mở rộng hoán vị E (Expansion permutation). Thực chất hàm mở rộng `E(Ri - 1)` là một hoán vị có lặp trong đó lặp lại 16 bit của Ri-1.
- Tính `E(Ri - 1) XOR Ki`.
- Tách kết quả của phép tính trên thành 8 xâu 6 bit `B1, B2, …, B8`.
- Đưa các khối 8 bit Bi vào 8 bảng `S1, S2, …, S8` (được gọi là các hộp S-box). Mỗi hộp S-Box là một bảng 4*16 cố định có các cột từ 0 đến 15 và các hàng từ 0 đến 3. Với mỗi xâu 6 bit `Bi = b1b2b3b4b5b6` ta tính được `SiBi` như sau: hai bit `b1b6` xác định hàng `r` trong hộp `Si`, bốn bit `b2b3b4b5` xác định cột `c` trong hộp `Si`. Khi đó, `Si(Bi)` sẽ xác định phần tử `Ci = Si(r,c)`, phần tử này viết dưới dạng nhị phân 4 bit. Như vậy, 8 khối 6 bit `Bi (1 <= i <= 8)` sẽ cho ra 8 khối 4 bit `Ci` với `(1 <= i <= 8)`.
- Xâu bit `C = C1C2C3C4C5C6C7C8` có độ dài 32 bit được hoán vị theo phép toán hoán vị P (hộp P-Box). Kết quả `P(C)` sẽ là kết quá của hàm `f(Ri - 1, Ki)`.

Hàm ở rộng E sẽ tăng độ dài `Ri - 1` từ 32 bit lên 48 bit bằng cách thay đổi thứ tự các bit cũng như lặp lại các bit. Việc thực hiện này nhằm hai mục đích:

Làm độ dài của `Ri - 1` cùng cỡ với khóa K để thực hiện việc cộng modulo `XOR`.

Cho kết quả dài hơn để có thể được nén trong suốt quá trình thay thế. Tuy nhiên, cả hai mục đích này nhằm một mục tiêu chính là bảo mật dữ liệu. Bằng cách cho phép 1 bit có thể chèn vào hai vị trí thay thế, sự phụ thuộc của các bit đầu ra với các bit đầu vào sẽ trải rộng ra. 

![PC1](../img/expand.jpg)

In [5]:
expansion_table = [
    32, 1,  2,  3,  4,  5,  4,  5, 
    6,  7,  8,  9,  8,  9,  10, 11,
    12, 13, 12, 13, 14, 15, 16, 17, 
    16, 17, 18, 19, 20, 21, 20, 21, 
    22, 23, 24, 25, 24, 25, 26, 27, 
    28, 29, 28, 29, 30, 31, 32, 1
]

def expand_r0(R0):
    expanded_r0 = [R0[i - 1] for i in expansion_table]  
    return expanded_r0

def bits_to_hex(bits):
    return hex(int(''.join(str(bit) for bit in bits), 2))[2:].zfill(12).upper()
def bits_to_hex_4bytes(bits):
    return hex(int(''.join(str(bit) for bit in bits), 2))[2:].zfill(8).upper()

r0_48 = expand_r0(R0)  

print(f"R0 (32 bit): {bits_to_hex_4bytes(R0)}")
print(f"R0 after Expansion P-box (48 bit): {bits_to_hex(r0_48)}")

R0 (32 bit): F0AAF0AA
R0 after Expansion P-box (48 bit): 7A15557A1555


R0 after Expansion P-box (48 bit) XOR K1 (48 bit)

In [7]:
def xor_bits(bits1, bits2):
    return [bit1 ^ bit2 for bit1, bit2 in zip(bits1, bits2)]
K1 = subkeys_48bit[0] 
xor_result = xor_bits(r0_48, K1)  
print(f"K1 XOR R0 after Ex P-box: {bits_to_hex(xor_result)}")

K1 XOR R0 after Ex P-box: 6117BA866527


48 bit đã XOR đi qua S-box thành 32 bit

Sau khi thực hiện phép `XOR` giữa `E(Ri - 1)` và `Ki`, kết quả thu được chuỗi 48 bit chia làm 8 khối đưa vào 8 hộp S-box. Mỗi hộp S-Box sẽ có 6 bit đầu vào và 4 bit đầu ra. Kết quả thu được là một chuỗi 32 bit tiếp tục vào hộp P-Box.

Mỗi hàng trong mỗi hộp S là hoán vị của các số nguyên từ 0 đến 15
Các hộp S-box phi tuyến tính. nói cách khác, đầu ra không phải là biến đối tuyến tính của đầu vào.
Sự thay đổi của một bit, hai bit hoặc nhiều hơn sẽ dẫn đến sự biến đổi ở đầu ra.
Nếu hai đầu vào của một S-box bất kì chỉ khác nhau 2 bit ở giữa (bit 3 và 4) thì đầu ra sẽ khác nhau ít nhất 2 bit. Nói cách khác, `S(x)` và `S(x XOR 001100)` phải khác nhau ít nhất 2 bit.

![PC1](../img/sbox1.webp)
![PC1](../img/sbox2.webp)
![PC1](../img/sbox3.webp)
![PC1](../img/sbox4.webp)
![PC1](../img/sbox5.webp)
![PC1](../img/sbox6.webp)
![PC1](../img/sbox7.webp)
![PC1](../img/sbox8.webp)

In [8]:
SBOX = [
    [
        [14, 4,  13, 1,  2,  15, 11, 8,  3,  10, 6,  12, 5,  9,  0,  7],
        [0,  15, 7,  4,  14, 2,  13, 1,  10, 6,  12, 11, 9,  5,  3,  8],
        [4,  1,  14, 8,  13, 6,  2,  11, 15, 12, 9,  7,  3,  10, 5,  0],
        [15, 12, 8,  2,  4,  9,  1,  7,  5,  11, 3,  14, 10, 0,  6,  13]
    ],
    [
        [15, 1,  8,  14, 6,  11, 3,  4,  9,  7,  2,  13, 12, 0,  5,  10],
        [3,  13, 4,  7,  15, 2,  8,  14, 12, 0,  1,  10, 6,  9,  11, 5],
        [0,  14, 7,  11, 10, 4,  13, 1,  5,  8,  12, 6,  9,  3,  2,  15],
        [13, 8,  10, 1,  3,  15, 4,  2,  11, 6,  7,  12, 0,  5,  14, 9]
    ],
    [
        [10, 0,  9,  14, 6,  3,  15, 5,  1,  13, 12, 7,  11, 4,  2,  8],
        [13, 7,  0,  9,  3,  4,  6,  10, 2,  8,  5,  14, 12, 11, 15, 1],
        [13, 6,  4,  9,  8,  15, 3,  0,  11, 1,  2,  12, 5,  10, 14, 7],
        [1,  10, 13, 0,  6,  9,  8,  7,  4,  15, 14, 3,  11, 5,  2,  12]
    ],
    [
        [7,  13, 14, 3,  0,  6,  9,  10, 1,  2,  8,  5,  11, 12, 4,  15],
        [13, 8,  11, 5,  6,  15, 0,  3,  4,  7,  2,  12, 1,  10, 14, 9],
        [10, 6,  9,  0,  12, 11, 7,  13, 15, 1,  3,  14, 5,  2,  8,  4],
        [3,  15, 0,  6,  10, 1,  13, 8,  9,  4,  5,  11, 12, 7,  2,  14]
    ],
    [
        [2,  12, 4,  1,  7,  10, 11, 6,  8,  5,  3,  15, 13, 0,  14, 9],
        [14, 11, 2,  12, 4,  7,  13, 1,  5,  0,  15, 10, 3,  9,  8,  6],
        [4,  2,  1,  11, 10, 13, 7,  8,  15, 9,  12, 5,  6,  3,  0,  14],
        [11, 8,  12, 7,  1,  14, 2,  13, 6,  15, 0,  9,  10, 4,  5,  3]
    ],
    [
        [12, 1,  10, 15, 9,  2,  6,  8,  0,  13, 3,  4,  14, 7,  5,  11],
        [10, 15, 4,  2,  7,  12, 9,  5,  6,  1,  13, 14, 0,  11, 3,  8],
        [9,  14, 15, 5,  2,  8,  12, 3,  7,  0,  4,  10, 1,  13, 11, 6],
        [4,  3,  2,  12, 9,  5,  15, 10, 11, 14, 1,  7,  6,  0,  8,  13]
    ],
    [
        [4,  11, 2,  14, 15, 0,  8,  13, 3,  12, 9,  7,  5,  10, 6,  1],
        [13, 0,  11, 7,  4,  9,  1,  10, 14, 3,  5,  12, 2,  15, 8,  6],
        [1,  4,  11, 13, 12, 3,  7,  14, 10, 15, 6,  8,  0,  5,  9,  2],
        [6,  11, 13, 8,  1,  4,  10, 7,  9,  5,  0,  15, 14, 2,  3,  12]
    ],
    [
        [13, 2,  8,  4,  6,  15, 11, 1,  10, 9,  3,  14, 5,  0,  12, 7],
        [1,  15, 13, 8,  10, 3,  7,  4,  12, 5,  6,  11, 0,  14, 9,  2],
        [7,  11, 4,  1,  9,  12, 14, 2,  0,  6,  10, 13, 15, 3,  5,  8],
        [2,  1,  14, 7,  4,  10, 8,  13, 15, 12, 9,  0,  3,  5,  6,  11]
    ]
]

def apply_sbox(bits_48):
    output_32 = []  
    for i in range(8):  
        sbox_input = bits_48[i*6:(i+1)*6]

        row = (sbox_input[0] << 1) | sbox_input[5] 
        col = (sbox_input[1] << 3) | (sbox_input[2] << 2) | (sbox_input[3] << 1) | sbox_input[4]  

        sbox_value = SBOX[i][row][col]

        output_32.extend([int(bit) for bit in bin(sbox_value)[2:].zfill(4)])

    return output_32

sbox_result = apply_sbox(xor_result)
print(f"S-box output (32 bit): {bits_to_hex(sbox_result)}")

S-box output (32 bit): 00005C82B597


Mỗi 4 bit đầu ra của các hộp S-box sẽ được ghép lại, theo thứ tự các hộp và được đem vào hộp P-box. Hộp P-Box đơn giản chỉ là hoán vị các bit với nhau.

![PC1](../img/pbox.webp)

In [ ]:
P_BOX = [
    16, 7,  20, 21, 29, 12, 28, 17,
    1,  15, 23, 26, 5,  18, 31, 10,
    2,  8,  24, 14, 32, 27, 3,  9,
    19, 13, 30, 6,  22, 11, 4,  25
]

def apply_pbox(bits):
    return [bits[i - 1] for i in P_BOX]

def bits_to_hex(bits):
    return hex(int(''.join(str(bit) for bit in bits), 2))[2:].zfill(8).upper()

pbox_result = apply_pbox(sbox_result)
print(f"P-box output (32 bit): {bits_to_hex(pbox_result)}")


P-box output (32 bit): 234AA9BB


`R1 = P-box output XOR L0`; `L1 = R0`

In [10]:
L0 = L0  
print(f"L0 (32-bit): {bits_to_hex(L0)}")

R0 = R0  
print(f"R0 (32-bit): {bits_to_hex(R0)}")

pbox_result = pbox_result  

L1 = R0

R1 = xor_bits(pbox_result, L0)


print(f"L1 (32-bit): {bits_to_hex(L1)}")
print(f"R1 (32-bit): {bits_to_hex(R1)}")

L0 (32-bit): CC00CCFF
R0 (32-bit): F0AAF0AA
L1 (32-bit): F0AAF0AA
R1 (32-bit): EF4A6544


Tìm `R2` tương tự `R1` (32 bit đi qua Expansion P-box -> 48 bit, `XOR K1` -> 48 bit, đi qua S-box -> 32 bit, đi qua P-box -> 32 bit, `XOR L1 -> R2`)

In [11]:
L2 = R1
R1_expanded = expand_r0(R1)  
print(f"R1 after Expansion (48-bit): {bits_to_hex(R1_expanded)}")

K2 = subkeys_48bit[1]  
xor_result = xor_bits(R1_expanded, K2)
print(f"R1 XOR K2 (48-bit): {bits_to_hex(xor_result)}")

sbox_result = apply_sbox(xor_result)  
print(f"S-box Output (32-bit): {bits_to_hex(sbox_result)}")

pbox_result = apply_pbox(sbox_result)
print(f"P-box Output (32-bit): {bits_to_hex(pbox_result)}")

R2 = xor_bits(pbox_result, L1)
print(f"R2 (32-bit): {bits_to_hex(R2)}")
print(f"L2 (32-bit): {bits_to_hex(L2)}")

R1 after Expansion (48-bit): 75EA5430AA09
R1 XOR K2 (48-bit): C448DEB63EC
S-box Output (32-bit): F8D03AAE
P-box Output (32-bit): 3CAB87A3
R2 (32-bit): CC017709
L2 (32-bit): EF4A6544


FULL SOURCE CODE:

In [ ]:
def hex2bin(s):
	mp = {'0': "0000",
		'1': "0001",
		'2': "0010",
		'3': "0011",
		'4': "0100",
		'5': "0101",
		'6': "0110",
		'7': "0111",
		'8': "1000",
		'9': "1001",
		'A': "1010",
		'B': "1011",
		'C': "1100",
		'D': "1101",
		'E': "1110",
		'F': "1111"}
	bin = ""
	for i in range(len(s)):
		bin = bin + mp[s[i]]
	return bin

def bin2hex(s):
	mp = {"0000": '0',
		"0001": '1',
		"0010": '2',
		"0011": '3',
		"0100": '4',
		"0101": '5',
		"0110": '6',
		"0111": '7',
		"1000": '8',
		"1001": '9',
		"1010": 'A',
		"1011": 'B',
		"1100": 'C',
		"1101": 'D',
		"1110": 'E',
		"1111": 'F'}
	hex = ""
	for i in range(0, len(s), 4):
		ch = ""
		ch = ch + s[i]
		ch = ch + s[i + 1]
		ch = ch + s[i + 2]
		ch = ch + s[i + 3]
		hex = hex + mp[ch]

	return hex

def bin2dec(binary):

	binary1 = binary
	decimal, i, n = 0, 0, 0
	while(binary != 0):
		dec = binary % 10
		decimal = decimal + dec * pow(2, i)
		binary = binary//10
		i += 1
	return decimal

def dec2bin(num):
	res = bin(num).replace("0b", "")
	if(len(res) % 4 != 0):
		div = len(res) / 4
		div = int(div)
		counter = (4 * (div + 1)) - len(res)
		for i in range(0, counter):
			res = '0' + res
	return res

def permute(k, arr, n):
	permutation = ""
	for i in range(0, n):
		permutation = permutation + k[arr[i] - 1]
	return permutation

def shift_left(k, nth_shifts):
	s = ""
	for i in range(nth_shifts):
		for j in range(1, len(k)):
			s = s + k[j]
		s = s + k[0]
		k = s
		s = ""
	return k

def xor(a, b):
	ans = ""
	for i in range(len(a)):
		if a[i] == b[i]:
			ans = ans + "0"
		else:
			ans = ans + "1"
	return ans

initial_perm = [58, 50, 42, 34, 26, 18, 10, 2,
				60, 52, 44, 36, 28, 20, 12, 4,
				62, 54, 46, 38, 30, 22, 14, 6,
				64, 56, 48, 40, 32, 24, 16, 8,
				57, 49, 41, 33, 25, 17, 9, 1,
				59, 51, 43, 35, 27, 19, 11, 3,
				61, 53, 45, 37, 29, 21, 13, 5,
				63, 55, 47, 39, 31, 23, 15, 7]

exp_d = [32, 1, 2, 3, 4, 5, 4, 5,
		6, 7, 8, 9, 8, 9, 10, 11,
		12, 13, 12, 13, 14, 15, 16, 17,
		16, 17, 18, 19, 20, 21, 20, 21,
		22, 23, 24, 25, 24, 25, 26, 27,
		28, 29, 28, 29, 30, 31, 32, 1]

per = [16, 7, 20, 21,
	29, 12, 28, 17,
	1, 15, 23, 26,
	5, 18, 31, 10,
	2, 8, 24, 14,
	32, 27, 3, 9,
	19, 13, 30, 6,
	22, 11, 4, 25]

sbox = [[[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
		[0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
		[4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
		[15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],

		[[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
		[3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
		[0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
		[13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],

		[[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
		[13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
		[13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
		[1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],

		[[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
		[13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
		[10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
		[3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],

		[[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
		[14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
		[4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
		[11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],

		[[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
		[10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
		[9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
		[4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],

		[[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
		[13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
		[1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
		[6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],

		[[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
		[1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
		[7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
		[2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]]

final_perm = [40, 8, 48, 16, 56, 24, 64, 32,
			39, 7, 47, 15, 55, 23, 63, 31,
			38, 6, 46, 14, 54, 22, 62, 30,
			37, 5, 45, 13, 53, 21, 61, 29,
			36, 4, 44, 12, 52, 20, 60, 28,
			35, 3, 43, 11, 51, 19, 59, 27,
			34, 2, 42, 10, 50, 18, 58, 26,
			33, 1, 41, 9, 49, 17, 57, 25]


def encrypt(pt, rkb, rk):
	pt = hex2bin(pt)

	pt = permute(pt, initial_perm, 64)
	print("init per", pt)
    # print("After initial permutation", bin2hex(pt))

	left = pt[0:32]
	right = pt[32:64]
	print('r', right)
	for i in range(0, 16):
		right_expanded = permute(right, exp_d, 48)
		print('right:', right_expanded)
		print('key', rkb[i])
		xor_x = xor(right_expanded, rkb[i])
		print('xor', xor_x)
		sbox_str = ""
		for j in range(0, 8):
			row = bin2dec(int(xor_x[j * 6] + xor_x[j * 6 + 5]))
			col = bin2dec(
				int(xor_x[j * 6 + 1] + xor_x[j * 6 + 2] + xor_x[j * 6 + 3] + xor_x[j * 6 + 4]))
			val = sbox[j][row][col]
			print(dec2bin(val))
			sbox_str = sbox_str + dec2bin(val)
		print('sbox', sbox_str)
		sbox_str = permute(sbox_str, per, 32)
		print('straight', sbox_str)
		result = xor(left, sbox_str)
		print('result ', result)
		left = result

		if(i != 15):
			left, right = right, left
		print("Round ", i + 1, " ", bin2hex(left),
    		" ", bin2hex(right), " ", rk[i])

	combine = left + right

	cipher_text = permute(combine, final_perm, 64)
	return cipher_text
	


# pt = "123456ABCD132536"
# key = "AABB09182736CCDD"
pt="0123456789ABCDEF"
key = "ABCD012345678910"

key = hex2bin(key)

keyp = [57, 49, 41, 33, 25, 17, 9,
		1, 58, 50, 42, 34, 26, 18,
		10, 2, 59, 51, 43, 35, 27,
		19, 11, 3, 60, 52, 44, 36,
		63, 55, 47, 39, 31, 23, 15,
		7, 62, 54, 46, 38, 30, 22,
		14, 6, 61, 53, 45, 37, 29,
		21, 13, 5, 28, 20, 12, 4]

key = permute(key, keyp, 56)

shift_table = [1, 1, 2, 2,
			2, 2, 2, 2,
			1, 2, 2, 2,
			2, 2, 2, 1]

key_comp = [14, 17, 11, 24, 1, 5,
			3, 28, 15, 6, 21, 10,
			23, 19, 12, 4, 26, 8,
			16, 7, 27, 20, 13, 2,
			41, 52, 31, 37, 47, 55,
			30, 40, 51, 45, 33, 48,
			44, 49, 39, 56, 34, 53,
			46, 42, 50, 36, 29, 32]

left = key[0:28] 
right = key[28:56] 

rkb = []
rk = []
for i in range(0, 16):
    left = shift_left(left, shift_table[i])
    right = shift_left(right, shift_table[i])
    # print("left", left, 'right', right)
    combine_str = left + right

    round_key = permute(combine_str, key_comp, 48)
    # print("\n")
    # print(round_key)

    rkb.append(round_key)
    rk.append(bin2hex(round_key))

print("Encryption")
cipher_text = bin2hex(encrypt(pt, rkb, rk))
print("Cipher Text : ", cipher_text)

print("Decryption")
rkb_rev = rkb[::-1]
rk_rev = rk[::-1]
text = bin2hex(encrypt(cipher_text, rkb_rev, rk_rev))
print("Plain Text : ", text)

# This code is contributed by Aditya Jain

Encryption
init per 1100110000000000110011001111111111110000101010101111000010101010
r 11110000101010101111000010101010
right: 011110100001010101010101011110100001010101010101
key 101110000101100000010100000000101100001000011001
xor 110000100100110101000001011110001101011101001100
1111
0111
1110
1101
1001
1001
1000
1011
sbox 11110111111011011001100110001011
straight 11111001100000111111101101010111
result  00110101100000110011011110101000
Round  1   F0AAF0AA   358337A8   B8581402C219
right: 000110101011110000000110100110101111110101010000
key 010001010101110000000010101100000000110010001010
xor 010111111110000000000100001010101111000111011010
1011
1111
1010
1110
1010
1010
0111
0000
sbox 10111111101011101010101001110000
straight 01010011111110000101011111010110
result  10100011010100101010011101111100
Round  2   358337A8   A352A77C   455C02B00C8A
right: 010100000110101010100101010100001110101111111001
key 011000101010000111010100000011000001001000010111
xor 00110010110010110111000101011